# 2단계: SageMaker AI 처리 및 훈련 작업 추가

<div class="alert alert-warning"> 이 노트북은 <code>SageMaker Distribution Image 3.6.1</code>을 사용하는 SageMaker Studio JupyterLab 인스턴스에서 SageMaker Python SDK 버전 <code>2.255.0</code>으로 마지막으로 테스트 되었습니다.</div>

이 단계에서는 데이터 처리와 모델 훈련을 [SageMaker Docker 컨테이너](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers.html)로 이동하고 [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/index.html)를 사용하여 SageMaker와 상호작용합니다.

||||
|---|---|---|
|1. |노트북에서 실험 ||
|2. |SageMaker AI 처리 작업과 SageMaker SDK로 확장 |**<<<< 현재 위치**|
|3. |ML 파이프라인, 모델 레지스트리, 피처 스토어로 운영화 ||
|4. |모델 빌드 CI/CD 파이프라인 추가 ||
|5. |모델 배포 파이프라인 추가 ||
|6. |모델 및 데이터 모니터링 추가 ||

SageMaker는 개발자가 데이터를 처리하고 모델을 훈련 및 배포할 수 있도록 Docker 컨테이너를 활용합니다. 컨테이너를 통해 개발자와 데이터 사이언티스트는 소프트웨어를 Docker를 지원하는 모든 플랫폼에서 일관되게 실행되는 표준화된 단위로 패키징할 수 있습니다. 컨테이너는 코드, 런타임, 시스템 도구, 시스템 라이브러리, 설정이 모두 같은 곳에 있도록 보장하여 실행 환경으로부터 격리시킵니다. 컨테이너가 어디서 실행되든 일관된 런타임 경험을 보장합니다.

SageMaker는 또한 인기 있는 데이터 처리 프레임워크와 ML 알고리즘이 포함된 사전 구축 컨테이너를 제공합니다. 모든 SageMaker 내장 알고리즘은 Docker 컨테이너로 제공됩니다.

<div class="alert alert-info"> 이 노트북에서는 JupyterLab에서 <code>Python 3</code> 커널을 사용하고 있는지 확인하세요.</div>

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
import time
import boto3
import botocore
import numpy as np  
import pandas as pd  
import sagemaker
import os
import mlflow
from time import gmtime, strftime, sleep
from sagemaker.processing import FrameworkProcessor, ProcessingInput, ProcessingOutput
from sagemaker.sklearn.estimator import SKLearn
from sklearn.metrics import roc_auc_score
from mlflow import MlflowClient
from IPython.display import Javascript
from importlib.metadata import version, PackageNotFoundError
from IPython.display import HTML

(sagemaker.__version__, boto3.__version__, mlflow.__version__)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
session = sagemaker.Session()
sm = session.sagemaker_client
training_job_name = None

## MLflow 실험 준비
이전 노트북에서 생성한 기존 실험을 재사용합니다. 동일한 실험에서 새로운 실행을 추적할 것입니다.
이 노트북에서 실행을 추적하기 위해 새로운 실험을 생성할 수도 있습니다 – 이 경우 다음 코드 셀의 주석을 해제하기만 하면 됩니다.

In [ ]:
experiment_suffix = strftime('%d-%H-%M-%S', gmtime())
registered_model_name = f"from-idea-to-prod-job-model-{experiment_suffix}"

In [ ]:
# 새로운 실험을 생성하려면 코드 블록의 주석을 해제하세요. (Cmd + /)
# experiment_name = f"from-idea-to-prod-experiment-{experiment_suffix}"

In [ ]:
print(f'Using MLflow server: {mlflow_arn}')
mlflow.set_tracking_uri(mlflow_arn)
experiment = mlflow.set_experiment(experiment_name=experiment_name)

## SageMaker 처리 작업으로 데이터 처리하기
Python 스크립트를 제공하고 [SageMaker SDK 프로세서](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_processing.html) 클래스를 선택하여 [SageMaker Processing](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)을 간단히 사용할 수 있습니다.

입력 데이터를 S3에 업로드하고 출력 데이터를 위한 S3 위치를 지정해야 합니다. SageMaker Processing은 자동으로 S3에서 입력 데이터를 로드하고 작업이 완료되면 변환된 데이터를 다시 S3에 업로드합니다. 처리 컨테이너 이미지는 Amazon SageMaker 내장 이미지이거나 사용자가 제공하는 사용자 정의 이미지일 수 있습니다. 처리 작업의 기본 인프라는 Amazon SageMaker에서 완전히 관리됩니다. 클러스터 리소스는 작업 기간 동안 프로비저닝되고 작업이 완료되면 정리됩니다.

![](img/sagemaker-processing.png)

이 섹션에서는 SageMaker 인프라를 사용하여 데이터 처리 스크립트를 실행하는 다양한 옵션을 보여줍니다:
1. 로컬 및 원격 모드에서 내장 SageMaker 컨테이너로 처리 실행
2. SageMaker Distribution Image 컨테이너를 사용하여 로컬 및 원격에서 처리 코드를 원격 함수로 실행

입력 데이터는 Amazon S3 버킷에 저장되어야 합니다. 또는 [Amazon Athena](https://sagemaker.readthedocs.io/en/stable/api/utility/inputs.html#sagemaker.dataset_definition.inputs.AthenaDatasetDefinition) 또는 [Amazon Redshift](https://sagemaker.readthedocs.io/en/stable/api/utility/inputs.html#sagemaker.dataset_definition.inputs.AthenaDatasetDefinition)를 입력 소스로 사용할 수 있습니다.

입력 데이터셋을 Amazon S3 버킷에 업로드하세요:

In [ ]:
input_s3_url = session.upload_data(
    path="data/bank-additional/bank-additional-full.csv",
    bucket=bucket_name,
    key_prefix=f"{bucket_prefix}/input"
)

%store input_s3_url

In [ ]:
# !aws s3 ls {bucket_name}/{bucket_prefix} --recursive

### SageMaker Framework 컨테이너에서 처리 스크립트 실행
로컬 모드에서 SageMaker 내장 관리 컨테이너로 데이터 처리 스크립트를 실행하는 것부터 시작합니다. [로컬 모드](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html)를 사용하면 로컬 개발 환경이나 SageMaker Space 인스턴스에서 SageMaker 작업을 실행할 수 있습니다. 이 기능은 SageMaker 원격 작업이나 파이프라인으로 실행하기 전에 원격 컴퓨팅 인스턴스를 생성하지 않고 스크립트를 테스트하고 디버깅하는 데 유용합니다.

SageMaker 내장 컨테이너나 자체 컨테이너(BYOC)에서 실행되는 스크립트에 로컬 모드를 사용할 수 있지만, XGBoost와 같은 일부 SageMaker 내장 알고리즘 컨테이너는 로컬 모드를 지원하지 않습니다.

<div class="alert alert-info">노트북 00에서 Docker를 설치했는지 확인하세요. 로컬 훈련을 건너뛰고 <b>2단계</b>로 바로 이동할 수도 있습니다.</div>

처리 스크립트에서 MLflow를 사용하고 있으므로, 처리 스크립트를 실행하기 전에 컨테이너가 mlflow와 MLflow SageMaker 플러그인을 설치할 수 있도록 `requirements.txt`를 제공해야 합니다.

In [ ]:
dependencies_dir="./processing/requirements/"
%mkdir -p processing
%mkdir -p {dependencies_dir}

In [ ]:
# 스크립트 컨테이너 환경에서 현재 패키지 버전 복제
packages = ['mlflow', 'sagemaker-mlflow']
requirements = [f'{p}=={version(p)}' for p in packages]

requirements.append('protobuf==3.20.2')

if requirements:
    with open(f'{dependencies_dir}/requirements.txt', 'w') as f:
        f.write('\n'.join(requirements))
    print("\nNew requirements.txt file created with the following content:")
    print('\n'.join(requirements))
else:
    print("\nNo requirements.txt file created as no packages were found")

1단계 노트북의 데이터 처리 코드를 .py 파일로 이동하고 MLflow를 사용한 실험 추적을 추가하여 Python 스크립트를 생성하세요:

In [ ]:
%%writefile processing/preprocessing.py

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import pandas as pd
import numpy as np
import argparse
import os

from time import gmtime, strftime, sleep
import traceback

user_profile_name = os.getenv('USER')

def _parse_args():
    
    parser = argparse.ArgumentParser()
    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default this is a S3 path under the default bucket.
    parser.add_argument('--filepath', type=str, default='/opt/ml/processing/input/')
    parser.add_argument('--filename', type=str, default='bank-additional-full.csv')
    parser.add_argument('--outputpath', type=str, default='/opt/ml/processing/output/')
    
    return parser.parse_known_args()

def process_data(df_data):
    # Indicator variable to capture when pdays takes a value of 999
    df_data["no_previous_contact"] = np.where(df_data["pdays"] == 999, 1, 0)

    # Indicator for individuals not actively employed
    df_data["not_working"] = np.where(
        np.in1d(df_data["job"], ["student", "retired", "unemployed"]), 1, 0
    )

    # remove unnecessary data
    df_model_data = df_data.drop(
        ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
        axis=1,
    )

    bins = [18, 30, 40, 50, 60, 70, 90]
    labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']

    df_model_data['age_range'] = pd.cut(df_model_data.age, bins, labels=labels, include_lowest=True)
    df_model_data = pd.concat([df_model_data, pd.get_dummies(df_model_data['age_range'], prefix='age', dtype=int)], axis=1)
    df_model_data.drop('age', axis=1, inplace=True)
    df_model_data.drop('age_range', axis=1, inplace=True)

    scaled_features = ['pdays', 'previous', 'campaign']
    df_model_data[scaled_features] = MinMaxScaler().fit_transform(df_model_data[scaled_features])

    df_model_data = pd.get_dummies(df_model_data, dtype=int)  # Convert categorical variables to sets of indicators

    # Replace "y_no" and "y_yes" with a single label column, and bring it to the front:
    df_model_data = pd.concat(
        [
            df_model_data["y_yes"].rename(target_col),
            df_model_data.drop(["y_no", "y_yes"], axis=1),
        ],
        axis=1,
    )
    
    return df_model_data

if __name__=="__main__":
    # Process arguments
    args, _ = _parse_args()
    
    target_col = "y"
    
    import mlflow
    
    # Set the Tracking Server URI using the ARN of the Tracking Server you created
    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_ARN'])
    
    # Enable autologging in MLflow
    mlflow.autolog()

    # Use the active run_id to log 
    with mlflow.start_run(run_id=os.environ['MLFLOW_RUN_ID']) as run:
        # process data
        df_model_data = process_data(pd.read_csv(os.path.join(args.filepath, args.filename), sep=";"))
    
        # Shuffle and splitting dataset
        train_data, validation_data, test_data = np.split(
            df_model_data.sample(frac=1, random_state=1729),
            [int(0.7 * len(df_model_data)), int(0.9 * len(df_model_data))],
        )
    
        print(f"Data split > train:{train_data.shape} | validation:{validation_data.shape} | test:{test_data.shape}")
        mlflow.log_params(
            {
                "train": train_data.shape,
                "validate": validation_data.shape,
                "test": test_data.shape
            }
        )

        mlflow.set_tags(
            {
                'mlflow.user':user_profile_name,
                'mlflow.source.type':'JOB'
            }
        )
        
        # Save datasets locally
        train_data.to_csv(os.path.join(args.outputpath, 'train/train.csv'), index=False, header=False)
        validation_data.to_csv(os.path.join(args.outputpath, 'validation/validation.csv'), index=False, header=False)
        test_data[target_col].to_csv(os.path.join(args.outputpath, 'test/test_y.csv'), index=False, header=False)
        test_data.drop([target_col], axis=1).to_csv(os.path.join(args.outputpath, 'test/test_x.csv'), index=False, header=False)
        
        # Save the baseline dataset for model monitoring
        df_model_data.drop([target_col], axis=1).to_csv(os.path.join(args.outputpath, 'baseline/baseline.csv'), index=False, header=False)

        mlflow.log_artifact(local_path=os.path.join(args.outputpath, 'baseline/baseline.csv'))
    
    print("## Processing complete. Exiting.")

처리 스크립트에는 헤더와 레이블 열 없이 전체 데이터셋을 베이스라인 데이터셋으로 저장하는 구문이 포함되어 있습니다. 이 데이터 베이스라인은 나중에 모델 모니터링 노트북에서 필요합니다.

Amazon S3 경로를 설정하세요:

In [ ]:
train_s3_url = f"s3://{bucket_name}/{bucket_prefix}/train"
validation_s3_url = f"s3://{bucket_name}/{bucket_prefix}/validation"
test_s3_url = f"s3://{bucket_name}/{bucket_prefix}/test"
baseline_s3_url = f"s3://{bucket_name}/{bucket_prefix}/baseline"

In [ ]:
%store train_s3_url
%store validation_s3_url
%store test_s3_url
%store baseline_s3_url

프레임워크 버전을 설정하세요:

In [ ]:
skprocessor_framework_version = '1.2-1'

#### 실험 실행 생성
처리 작업의 매개변수, 구성, 입력 및 출력을 추적하기 위해 실험에서 새로운 실행을 생성하세요.

In [ ]:
run_suffix = strftime('%d-%H-%M-%S', gmtime())
run_name = f"container-processing-{run_suffix}"

run_id = mlflow.start_run(
    run_name=run_name,
    description="feature-engineering in the notebook 02 with a processing job").info.run_id

#### 프로세서 생성 및 입력과 출력 설정
SageMaker 처리 작업을 시작하기 전에 [FrameworkProcessor](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks.html) 객체를 인스턴스화하세요. 로컬 모드를 활성화하기 위해 인스턴스 타입을 `local`로 설정하고 `sagemaker_session` 매개변수로 `sagemaker.LocalSession()` 객체를 제공하세요.

SageMaker가 데이터를 처리 컨테이너의 EBS 볼륨의 로컬 경로에 매핑하는 방법을 확인하세요:

![](img/data-processing.png)

In [ ]:
sklearn_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version=skprocessor_framework_version,
    role=sm_role,
    instance_type='local',
    instance_count=1,
    base_job_name='from-idea-to-prod-processing',
    sagemaker_session=sagemaker.LocalSession(), 
    env={
        'MLFLOW_TRACKING_ARN': mlflow_arn,
        'MLFLOW_RUN_ID': run_id,
        'USER': user_profile_name
    }
)

processing_inputs = [
        ProcessingInput(
            source=input_s3_url, 
            destination="/opt/ml/processing/input",
            # s3_input_mode="File",
            # s3_data_distribution_type="ShardedByS3Key"
        ),
        ProcessingInput(
            input_name="processor",
            source=dependencies_dir,
            destination="/opt/ml/processing/input/code/requirements/",
        ),
    ]

processing_outputs = [
        ProcessingOutput(
            output_name="train_data", 
            source="/opt/ml/processing/output/train",
            destination=train_s3_url,
        ),
        ProcessingOutput(
            output_name="validation_data", 
            source="/opt/ml/processing/output/validation", 
            destination=validation_s3_url
        ),
        ProcessingOutput(
            output_name="test_data", 
            source="/opt/ml/processing/output/test", 
            destination=test_s3_url
        ),
        ProcessingOutput(
            output_name="baseline_data", 
            source="/opt/ml/processing/output/baseline", 
            destination=baseline_s3_url
        ),
    ]

<div class="alert alert-info">아래 코드에서 <code>No such file or directory: 'docker'</code> 오류가 발생하면, 노트북 00의 <b>Studio 로컬 모드를 활성화하기 위한 Docker 설치</b> 섹션을 실행해야 합니다.</div>

#### SageMaker 처리 작업 시작



프로세서가 처음 실행될 때는 docker를 ECR에 인증하고 기본 SageMaker sklearn 이미지를 가져옵니다. 이후 프로세서 실행은 더 빠릅니다.

In [ ]:
sklearn_processor.run(
    inputs=processing_inputs,
    outputs=processing_outputs,
    code='processing/preprocessing.py',
    dependencies=[f'{dependencies_dir}requirements.txt'],
    # arguments = ['arg1', 'arg2'],
)

mlflow.set_tags(
    {
        'mlflow.source.name':f'https://{region}.console.aws.amazon.com/sagemaker/home?region={region}#/processing-jobs/{sklearn_processor.latest_job.name}',
    }
)

mlflow.end_run()

In [ ]:
# check that the processing script uploaded the dataset to the S3 prefix
!aws s3 ls {train_s3_url}/

### 처리 스크립트를 로컬 및 원격에서 SageMaker 작업으로 실행
이제 [SageMaker Python SDK 데코레이터 @remote](https://sagemaker.readthedocs.io/en/stable/remote_function/sagemaker.remote_function.html)를 사용하여 노트북의 로컬 코드를 SageMaker 처리 작업으로 실행합니다. 이를 "원격 함수"라고 합니다. 이는 SageMaker 분산 처리 및 훈련을 사용하여 Python 코드를 대규모로 실행하는 더욱 쉬운 방법입니다. Amazon SageMaker 개발자 가이드의 [문서](https://docs.aws.amazon.com/sagemaker/latest/dg/train-remote-decorator.html)를 참조하세요.

처리 작업을 실행하는 또 다른 옵션은 [RemoteExecutor](https://sagemaker.readthedocs.io/en/stable/remote_function/sagemaker.remote_function.html#remoteexecutor)를 사용하는 것입니다. 이는 로컬 함수를 SageMaker 작업으로 원격 실행할 수 있게 해주는 SageMaker Python SDK 클래스입니다. `max_parallel_jobs` 매개변수를 사용하여 최대 병렬 작업 수를 제어하면서 여러 작업을 병렬로 실행할 수 있습니다.

다음 섹션에서는 `@remote` 데코레이터와 RemoteExecutor API를 모두 사용하여 데이터 처리 코드를 SageMaker 작업으로 실행합니다.



#### 1단계: 로컬에서 코드 개발 및 테스트
먼저 노트북에서 로컬로 코드를 구현하고 테스트하여 코드와 환경의 정확성을 확인합니다.

In [ ]:
# Load the dataset in to DataFrame
file_name = "bank-additional-full.csv"
input_path = "./data/bank-additional" 
df_data = pd.read_csv(os.path.join(input_path, file_name), sep=";")

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# define a local function
def preprocess(
    df_data,
    tracking_server_arn=mlflow_arn,
    experiment_name=None,
    run_id=None,
):
    import mlflow
    from time import gmtime, strftime

    try:
        # Set the Tracking Server URI using the ARN of the Tracking Server you created
        mlflow.set_tracking_uri(tracking_server_arn)
        
        # Enable autologging in MLflow
        mlflow.autolog()
    
        suffix = strftime('%d-%H-%M-%S', gmtime())
        mlflow.set_experiment(experiment_name=experiment_name if experiment_name else f"preprocess-{suffix}")
        run = mlflow.start_run(run_id=run_id) if run_id else mlflow.start_run(run_name=f"local-processing-{suffix}", nested=True)
    
        target_col = "y"
        
        # Indicator variable to capture when pdays takes a value of 999
        df_data["no_previous_contact"] = np.where(df_data["pdays"] == 999, 1, 0)
    
        # Indicator for individuals not actively employed
        df_data["not_working"] = np.where(
            np.in1d(df_data["job"], ["student", "retired", "unemployed"]), 1, 0
        )
    
        # remove unnecessary data
        df_model_data = df_data.drop(
            ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
            axis=1,
        )
    
        bins = [18, 30, 40, 50, 60, 70, 90]
        labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']
    
        df_model_data['age_range'] = pd.cut(df_model_data.age, bins, labels=labels, include_lowest=True)
        df_model_data = pd.concat([df_model_data, pd.get_dummies(df_model_data['age_range'], prefix='age', dtype=int)], axis=1)
        df_model_data.drop('age', axis=1, inplace=True)
        df_model_data.drop('age_range', axis=1, inplace=True)
    
        scaled_features = ['pdays', 'previous', 'campaign']
        df_model_data[scaled_features] = MinMaxScaler().fit_transform(df_model_data[scaled_features])
    
        df_model_data = pd.get_dummies(df_model_data, dtype=int)  # Convert categorical variables to sets of indicators
    
        # Replace "y_no" and "y_yes" with a single label column, and bring it to the front:
        df_model_data = pd.concat(
            [
                df_model_data["y_yes"].rename(target_col),
                df_model_data.drop(["y_no", "y_yes"], axis=1),
            ],
            axis=1,
        )
    
        # Shuffle and splitting dataset
        train_data, validation_data, test_data = np.split(
            df_model_data.sample(frac=1, random_state=1729),
            [int(0.7 * len(df_model_data)), int(0.9 * len(df_model_data))],
        )
    
        print(f"Data split > train:{train_data.shape} | validation:{validation_data.shape} | test:{test_data.shape}")

        mlflow.log_params(
            {
                "train": train_data.shape,
                "validate": validation_data.shape,
                "test": test_data.shape
            }
        )
        
        baseline_data = df_model_data.drop([target_col], axis=1)
        
        print("## Processing complete. Exiting.")
        
        return train_data, validation_data, test_data, baseline_data

    except Exception as e:
        print(f"Exception in processing script: {e}")
        raise e
    finally:
        mlflow.end_run()

In [ ]:
# Call the function locally
train_data, validation_data, test_data, baseline_data = preprocess(df_data, experiment_name=experiment_name)

In [ ]:
# see the processed data
train_data.head()

#### 2단계: RemoteExecutor를 사용하여 함수를 원격으로 실행

In [ ]:
from sagemaker.remote_function import remote, RemoteExecutor

In [ ]:
s3_root_uri = f"s3://{bucket_name}/{bucket_prefix}"

In [ ]:
# this code will start a SageMaker job to execute prerpocess script
with RemoteExecutor(dependencies=f"{dependencies_dir}requirements.txt",
                    s3_root_uri=s3_root_uri, instance_type='ml.m5.xlarge') as e:
    future = e.submit(preprocess, df_data)                 

In [ ]:
# the call future.results() fetches the job execution logs and deserializes the script output back to the objects
train_data, validation_data, test_data, baseline_data = future.result()

In [ ]:
# see the processed data
train_data.head()

#### 3단계: @remote 데코레이터로 코드 실행
[구성 파일](https://docs.aws.amazon.com/sagemaker/latest/dg/train-remote-decorator-config.html)을 통해 원격 함수의 기본 설정을 지정할 수도 있습니다. 구성 파일은 `@remote` 데코레이터나 `RemoteExecutor` API로 함수를 호출할 때 사용됩니다. 노트북 3에서 SageMaker 파이프라인을 구성하기 위해 SageMaker 구성 파일을 사용할 예정입니다.

<div class="alert alert-info">이는 선택적 단계로, <code>@remote</code> 데코레이터를 사용하여 로컬 코드를 원격 함수로 이전하는 또 다른 방법을 보여줍니다. 시간이 부족하다면 이 섹션을 실행할 필요가 없습니다.</div>

In [ ]:
@remote(dependencies=f"{dependencies_dir}requirements.txt",
        s3_root_uri=s3_root_uri, instance_type='ml.m5.xlarge')
def preprocess(
    df_data,
    tracking_server_arn=mlflow_arn,
    experiment_name=None,
    run_id=None,
):
    import mlflow
    from time import gmtime, strftime

    try:
        # Set the Tracking Server URI using the ARN of the Tracking Server you created
        mlflow.set_tracking_uri(tracking_server_arn)
        
        # Enable autologging in MLflow
        mlflow.autolog()
    
        suffix = strftime('%d-%H-%M-%S', gmtime())
        mlflow.set_experiment(experiment_name=experiment_name if experiment_name else f"preprocess-{suffix}")
        run = mlflow.start_run(run_id=run_id) if run_id else mlflow.start_run(run_name=f"remote-processing-{suffix}", nested=True)
    
        target_col = "y"
        
        # Indicator variable to capture when pdays takes a value of 999
        df_data["no_previous_contact"] = np.where(df_data["pdays"] == 999, 1, 0)
    
        # Indicator for individuals not actively employed
        df_data["not_working"] = np.where(
            np.in1d(df_data["job"], ["student", "retired", "unemployed"]), 1, 0
        )
    
        # remove unnecessary data
        df_model_data = df_data.drop(
            ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
            axis=1,
        )
    
        bins = [18, 30, 40, 50, 60, 70, 90]
        labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']
    
        df_model_data['age_range'] = pd.cut(df_model_data.age, bins, labels=labels, include_lowest=True)
        df_model_data = pd.concat([df_model_data, pd.get_dummies(df_model_data['age_range'], prefix='age', dtype=int)], axis=1)
        df_model_data.drop('age', axis=1, inplace=True)
        df_model_data.drop('age_range', axis=1, inplace=True)
    
        scaled_features = ['pdays', 'previous', 'campaign']
        df_model_data[scaled_features] = MinMaxScaler().fit_transform(df_model_data[scaled_features])
    
        df_model_data = pd.get_dummies(df_model_data, dtype=int)  # Convert categorical variables to sets of indicators
    
        # Replace "y_no" and "y_yes" with a single label column, and bring it to the front:
        df_model_data = pd.concat(
            [
                df_model_data["y_yes"].rename(target_col),
                df_model_data.drop(["y_no", "y_yes"], axis=1),
            ],
            axis=1,
        )
    
        # Shuffle and splitting dataset
        train_data, validation_data, test_data = np.split(
            df_model_data.sample(frac=1, random_state=1729),
            [int(0.7 * len(df_model_data)), int(0.9 * len(df_model_data))],
        )
    
        print(f"Data split > train:{train_data.shape} | validation:{validation_data.shape} | test:{test_data.shape}")

        mlflow.log_params(
            {
                "train": train_data.shape,
                "validate": validation_data.shape,
                "test": test_data.shape
            }
        )
        
        baseline_data = df_model_data.drop([target_col], axis=1)
        
        print("## Processing complete. Exiting.")
        
        return train_data, validation_data, test_data, baseline_data

    except Exception as e:
        print(f"Exception in processing script: {e}")
        raise e
    finally:
        mlflow.end_run()
    return train_data, validation_data, test_data, baseline_data

In [ ]:
# This call creates and run a SageMaker job
# This will also create a new experiment in MLflow
train_data, validation_data, test_data, baseline_data = preprocess(df_data, experiment_name=experiment_name)

In [ ]:
# see the processed data
train_data.head()

For more examples of remote functions see SageMaker [example notebooks](https://docs.aws.amazon.com/sagemaker/latest/dg/train-remote-decorator-examples.html).

## SageMaker 훈련 작업을 통한 모델 훈련
처리 작업과 동일한 접근 방식을 따라 스크립트 모드에서 내장 프레임워크 컨테이너를 사용하여 [SageMaker 훈련 작업](https://sagemaker.readthedocs.io/en/stable/overview.html#using-estimators)으로 모델 훈련을 실행할 수 있습니다. 스크립트 모드는 SageMaker에서 미리 정의된 관리형 컨테이너 중 하나에서 실행할 스크립트를 제공하는 것을 의미합니다. 이 접근 방식을 통해 SageMaker에서 제공하는 표준 실행 환경을 사용하면서 코드 개발에 집중할 수 있습니다. 또한 Studio JupyterLab 공간에서 Docker 컨테이너로 코드를 로컬에서 먼저 실행한 다음 SageMaker 원격 작업으로 이동할 수도 있습니다.

다음 그림은 SageMaker 훈련 옵션을 요약합니다:

![](img/sagemaker-training-options.png)

In [ ]:
!mkdir -p ./training/
!sudo rm -rf ./training-local/
!mkdir -p ./training-local/

In [ ]:
# check the XGBoost version
%pip show xgboost

### 훈련 스크립트 준비
훈련 스크립트가 포함된 `train.py` 파일과 환경 구성이 포함된 `requirements.txt` 파일을 생성합니다.

In [ ]:
%%writefile ./training/train.py

import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl

from sagemaker_containers import entry_point
from sagemaker_xgboost_container.data_utils import get_dmatrix
from sagemaker_xgboost_container import distributed

from sklearn.metrics import roc_auc_score

import xgboost as xgb

from time import gmtime, strftime

suffix = strftime('%d-%H-%M-%S', gmtime())

user_profile_name = os.getenv('USER', 'sagemaker')
experiment_name = os.getenv('MLFLOW_EXPERIMENT_NAME')
region = os.getenv('REGION')

def _xgb_train(params, dtrain, dval, evals, num_boost_round, model_dir, is_master):
    """Run xgb train on arguments given with rabit initialized.

    This is our rabit execution function.

    :param args_dict: Argument dictionary used to run xgb.train().
    :param is_master: True if current node is master host in distributed training,
                        or is running single node training job.
                        Note that rabit_run includes this argument.
    """
    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        evals=evals,
        num_boost_round=num_boost_round
    )

    val_auc = roc_auc_score(dval.get_label(), booster.predict(dval))
    train_auc = roc_auc_score(dtrain.get_label(), booster.predict(dtrain))
    mlflow.log_params(params)
    mlflow.log_metrics({"validation_auc":val_auc, "train_auc":train_auc})
    # emit training metrics - SageMaker collects them from the log stream
    print(f"[0]#011train-auc:{train_auc}#011validation-auc:{val_auc}")
    
    if is_master:
        model_location = model_dir + '/xgboost-model'
        pkl.dump(booster, open(model_location, 'wb'))
        print("Stored trained model at {}".format(model_location))


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    # Hyperparameters are described here.
    parser.add_argument('--max_depth', type=int)
    parser.add_argument('--eta', type=float)
    parser.add_argument('--alpha', type=float)
    parser.add_argument('--gamma', type=int)
    parser.add_argument('--min_child_weight', type=float)
    parser.add_argument('--subsample', type=float)
    parser.add_argument('--colsample_bytree', type=float)
    parser.add_argument('--verbosity', type=int)
    parser.add_argument('--objective', type=str)
    parser.add_argument('--num_round', type=int)
    parser.add_argument('--early_stopping_rounds', type=int)
    parser.add_argument('--tree_method', type=str, default="auto")
    parser.add_argument('--predictor', type=str, default="auto")

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output_data_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--sm_hosts', type=str, default=os.environ.get('SM_HOSTS'))
    parser.add_argument('--sm_current_host', type=str, default=os.environ.get('SM_CURRENT_HOST'))
    parser.add_argument('--sm_training_env', type=str, default=os.environ.get('SM_TRAINING_ENV'))
    
    print("main function")
    args, _ = parser.parse_known_args()

    # Get SageMaker host information from runtime environment variables
    sm_hosts = json.loads(args.sm_hosts)
    sm_current_host = args.sm_current_host
    dtrain = get_dmatrix(args.train, 'CSV')
    dval = get_dmatrix(args.validation, 'CSV')

    watchlist = [(dtrain, 'train'), (dval, 'validation')] if dval is not None else [(dtrain, 'train')]

    # get SageMaker enviroment setup
    sm_training_env = json.loads(args.sm_training_env)

    import mlflow
    mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_ARN'))
    mlflow.set_experiment(experiment_name=experiment_name if experiment_name else f"train-{suffix}")
    
    # enable auto logging
    mlflow.xgboost.autolog(log_model_signatures=False, log_datasets=False)

    train_hp = {
        'max_depth': args.max_depth,
        'eta': args.eta,
        'gamma': args.gamma,
        'min_child_weight': args.min_child_weight,
        'subsample': args.subsample,
        'verbosity': args.verbosity,
        'objective': args.objective,
        'tree_method': args.tree_method,
        'predictor': args.predictor,
    }

    xgb_train_args = dict(
        params=train_hp,
        dtrain=dtrain,
        dval=dval,
        evals=watchlist,
        num_boost_round=args.num_round,
        model_dir=args.model_dir)

    with mlflow.start_run(
        run_name=f"container-training-{suffix}",
        description="xgboost running in SageMaker container in script mode"
    ) as run:

        mlflow.set_tags(
            {
                'mlflow.user':user_profile_name,
                'mlflow.source.type':'JOB',
                'mlflow.source.name': f"https://{region}.console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{sm_training_env['job_name']}" if sm_training_env['current_host'] != 'sagemaker-local' else sm_training_env['current_host']
            }
        )
    
        if len(sm_hosts) > 1:
            # Wait until all hosts are able to find each other
            entry_point._wait_hostname_resolution()
    
            # Execute training function after initializing rabit.
            distributed.rabit_run(
                exec_fun=_xgb_train,
                args=xgb_train_args,
                include_in_training=(dtrain is not None),
                hosts=sm_hosts,
                current_host=sm_current_host,
                update_rabit_args=True
            )
        else:
            # If single node training, call training method directly.
            if dtrain:
                xgb_train_args['is_master'] = True
                _xgb_train(**xgb_train_args)
            else:
                raise ValueError("Training channel must have data to train model.")

# Return model object
def model_fn(model_dir):
    """Deserialize and return fitted model.

    Note that this should have the same name as the serialized model in the _xgb_train method
    """
    model_file = 'xgboost-model'
    booster = pkl.load(open(os.path.join(model_dir, model_file), 'rb'))
    return booster

이전 처리 작업과 마찬가지로, 훈련 작업을 위해 MLflow 종속성을 설치할 요구사항 파일을 준비합니다.

In [ ]:
# replicate the current package versions in the script container environment
packages = ['mlflow', 'sagemaker-mlflow']
requirements = [f'{p}=={version(p)}' for p in packages]

requirements.append('protobuf==3.20.1')

if requirements:
    with open(f'./training/requirements.txt', 'w') as f:
        f.write('\n'.join(requirements))
    print("\nNew requirements.txt file created with the following content:")
    print('\n'.join(requirements))
else:
    print("\nNo requirements.txt file created as no packages were found")

### 데이터 및 하이퍼파라미터 준비
훈련 작업을 위한 데이터 입력 채널을 정의합니다. SageMaker SDK [`TrainingInput`](https://sagemaker.readthedocs.io/en/stable/api/utility/inputs.html#sagemaker.inputs.TrainingInput) 클래스를 통해 _train_ 및 _validation_ 채널을 설정합니다:

In [ ]:
s3_input_train = sagemaker.inputs.TrainingInput(train_s3_url, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(validation_s3_url, content_type='csv')

training_inputs = {'train': s3_input_train, 'validation': s3_input_validation}

In [ ]:
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

# Define where the training job stores the model artifact
output_s3_url = f"s3://{bucket_name}/{bucket_prefix}/output"

%store output_s3_url

환경 변수에서 하이퍼파라미터와 MLflow 설정을 준비합니다:

In [ ]:
hyperparams = {
    'num_round': 50,
    'max_depth': 3,
    'eta': 0.5,
    'alpha': 2.5,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'early_stopping_rounds': 10,
    'verbosity': 1
}

env_variables = {
    'MLFLOW_TRACKING_ARN': mlflow_arn,
    'MLFLOW_EXPERIMENT_NAME': experiment_name,
    'USER': user_profile_name,
    'REGION': region,
}

### 1단계: 로컬 모드에서 훈련 실행
SageMaker를 로컬 모드에서 실행하는 것은 노트북에서 훈련 스크립트가 의도한 대로 작동하는지 확인하기 위해 빠르게 반복 작업을 수행할 수 있는 편리한 방법입니다.

훈련 스크립트를 다른 폴더(예: `./training-local/`)로 복사하여 컨테이너에서 로컬로 스크립트를 테스트할 수 있습니다. 이렇게 하면 SageMaker 관리형 인프라를 구동할 필요 없이 스크립트를 빠르게 개발할 수 있어 시간과 비용을 절약할 수 있습니다.

In [ ]:
%cp -rf ./training/* ./training-local

SageMaker 관리형 작업으로 훈련 작업을 실행하는 것과 동일한 코드를 사용하지만, `sagemaker_session` 매개변수를 `LocalSession`으로, `instance_type`을 'local'로 설정합니다:

In [ ]:
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.local import LocalSession

LOCAL_SESSION = LocalSession()
LOCAL_SESSION.config = {'local': {'local_code': True}}  # Ensure full code locality, see: https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode

xgb_script_mode_local = XGBoost(
    entry_point='train.py',
    source_dir='./training-local',
    framework_version="1.7-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=sm_role,
    instance_count=train_instance_count,
    instance_type='local',
    output_path=output_s3_url,
    base_job_name="from-idea-to-prod-training",
    environment=env_variables,
    sagemaker_session=LOCAL_SESSION,
)

<div style="border: 4px solid coral; text-align: center; margin: auto;">
XGBoost 컨테이너가 로컬로 다운로드되기 때문에 로컬에서 훈련을 처음 실행할 때는 시간이 다소 걸릴 수 있습니다.
</div>

<div class="alert alert-info">아래 코드에서 <code>No such file or directory: 'docker'</code> 오류가 발생하면, 노트북 00의 <b>Studio 로컬 모드를 활성화하기 위한 Docker 설치</b> 섹션을 실행해야 합니다.</div>

In [ ]:
xgb_script_mode_local.fit(
    training_inputs,
    wait=True,
    logs=True,
)

훈련이 완료된 후 MLflow 실험에서 결과, 메트릭 및 모델을 확인할 수 있습니다. 해당 MLflow 실험 및 실행에 대한 링크를 구성해보겠습니다.

In [ ]:
# get the last run in MLflow
last_run_id = mlflow.search_runs(
    experiment_ids=[mlflow.get_experiment_by_name(experiment_name).experiment_id], 
    max_results=1, 
    order_by=["attributes.start_time DESC"]
)['run_id'][0]

# get the presigned url to open the MLflow UI
presigned_url = sm.create_presigned_mlflow_tracking_server_url(
    TrackingServerName=mlflow_name,
    ExpiresInSeconds=60,
    SessionExpirationDurationInSeconds=1800
)['AuthorizedUrl']

mlflow_run_link = f"{presigned_url.split('/auth')[0]}/#/experiments/1/runs/{last_run_id}"

In [ ]:
# first open the MLflow UI - you can close a new opened window
display(Javascript('window.open("{}");'.format(presigned_url)))

In [ ]:
# second open the run page in the MLflow UI
display(Javascript('window.open("{}");'.format(mlflow_run_link)))

### 2단계: 스크립트 모드에서 SageMaker 훈련 작업으로 훈련 실행
동일한 코드를 사용하여 SageMaker 관리형 인프라에서 훈련 스크립트를 원격으로 실행합니다. `LocalSession`을 제거하고 `instance_type`을 `local`에서 원하는 컴퓨팅 인스턴스로 변경합니다.

In [ ]:
from sagemaker.xgboost.estimator import XGBoost

xgb_script_mode_managed = XGBoost(
    entry_point='train.py',
    source_dir='./training',
    framework_version="1.7-1",  
    hyperparameters=hyperparams,
    role=sm_role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    output_path=output_s3_url,
    base_job_name="from-idea-to-prod-training",
    environment=env_variables,
)

In [ ]:
xgb_script_mode_managed.fit(
    training_inputs,
    wait=True,
    logs=False,
)

In [ ]:
# Collect the emitted metrics from the log stream
xgb_script_mode_managed.training_job_analytics.dataframe()

SageMaker 콘솔에서 모든 작업 세부 정보를 확인할 수 있습니다:

In [ ]:
# Show the training job link
display(
    HTML('<b>See the SageMaker <a target="top" href="https://{}.console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">training job</a></b>'.format(
            region, region, xgb_script_mode_managed.latest_training_job.name))
)

Click on the link ^^^ above ^^^ to open the SageMaker console with the training job details.

### 선택사항: SageMaker 내장 알고리즘으로 훈련
자체 스크립트를 개발하는 대신 SageMaker [내장 알고리즘](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html) 중 하나를 사용할 수 있습니다. 이 섹션에서는 [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 알고리즘을 사용합니다. 자체 훈련 스크립트를 제공할 필요가 없습니다. [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) 객체를 인스턴스화하고, 알고리즘의 [하이퍼파라미터](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)를 설정한 다음, 추정기의 `fit()` 메서드를 호출하기만 하면 됩니다.

<div class="alert alert-info">이는 선택적 단계로, 훈련 스크립트를 작성하지 않고 SageMaker 내장 알고리즘을 사용하여 모델을 훈련하는 방법을 보여줍니다. 시간이 제한적이라면 이 섹션을 실행할 필요가 없습니다.</div>

In [ ]:
# get training container uri
training_image = sagemaker.image_uris.retrieve("xgboost", region=region, version="1.7-1")

print(training_image)

In [ ]:
# Instantiate an XGBoost estimator object
estimator = sagemaker.estimator.Estimator(
    image_uri=training_image,  # XGBoost algorithm container
    instance_type=train_instance_type,  # type of training instance
    instance_count=train_instance_count,  # number of instances to be used
    role=sm_role,  # IAM execution role to be used
    max_run=20 * 60,  # Maximum allowed active runtime
    # use_spot_instances=True,  # Use spot instances to reduce cost
    # max_wait=30 * 60,  # Maximum clock time (including spot delays)
    output_path=output_s3_url, # S3 location for saving the training result
    sagemaker_session=session, # Session object which manages interactions with SageMaker API and AWS services
    base_job_name="from-idea-to-prod-training", # Prefix for training job name
)

# define its hyperparameters
estimator.set_hyperparameters(
    num_round=50, # the number of rounds to run the training
    max_depth=3, # maximum depth of a tree
    eta=0.5, # step size shrinkage used in updates to prevent overfitting
    alpha=2.5, # L1 regularization term on weights
    objective="binary:logistic",
    eval_metric="auc", # evaluation metrics for validation data
    subsample=0.8, # subsample ratio of the training instance
    colsample_bytree=0.8, # subsample ratio of columns when constructing each tree
    min_child_weight=3, # minimum sum of instance weight (hessian) needed in a child
    early_stopping_rounds=10, # the model trains until the validation score stops improving
    verbosity=1, # verbosity of printing messages
)

In [ ]:
estimator.hyperparameters()

In [ ]:
# helper function to load XGBoost model into xgboost.Booster
def load_model(model_data_s3_uri):
    import xgboost as xgb
    import tarfile
    import pickle as pkl

    model_file = "./xgboost-model.tar.gz"
    bucket, key = model_data_s3_uri.replace("s3://", "").split("/", 1)
    boto3.client("s3").download_file(bucket, key, model_file)
    
    with tarfile.open(model_file, "r:gz") as t:
        t.extractall(path=".")
    
    # Load model
    model = xgb.Booster()
    model.load_model("xgboost-model")

    return model

훈련을 실행합니다:

In [ ]:
mlflow.set_experiment(experiment_name=experiment_name)
with mlflow.start_run(
    run_name=f"container-training-{strftime('%d-%H-%M-%S', gmtime())}",
    description="training in the notebook 02 with a training job") as run:
    mlflow.log_params(estimator.hyperparameters())
        
    estimator.fit(
        training_inputs,
        wait=True,
        logs=False,
    ) 
    
    mlflow.set_tags(
        {
            'mlflow.user':user_profile_name,
            'mlflow.source.name':f'https://{region}.console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{estimator.latest_training_job.name}',
            'mlflow.source.type':'JOB'
        }
    )
    mlflow.log_param("training job name", estimator.latest_training_job.name)
    mlflow.log_metrics({i['metric_name'].replace(':', '_'):i['value'] for i in estimator.training_job_analytics.dataframe().iloc})
    mlflow.xgboost.log_model(load_model(estimator.model_data), artifact_path="model")

아래 셀에서 생성된 링크를 클릭하여 훈련 작업의 세부 정보를 확인합니다:

In [ ]:
# Show the training job link
display(
    HTML('<b>See the SageMaker <a target="top" href="https://{}.console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">training job</a></b>'.format(
            region, region, estimator.latest_training_job.name))
)

#### 모델 성능 출력

In [ ]:
if estimator._current_job_name:
    training_job_name = estimator._current_job_name

In [ ]:
%store training_job_name

In [ ]:
metrics = None
while not metrics:
    metrics = sm.describe_training_job(
        TrainingJobName=training_job_name
        ).get("FinalMetricDataList")

    if not metrics:
        print(f"Training job {training_job_name} hasn't finished yet!")
        time.sleep(10)
    
train_auc = float([m['Value'] for m in metrics if m['MetricName'] == 'train:auc'][0])
validate_auc = float([m['Value'] for m in metrics if m['MetricName'] == 'validation:auc'][0])

print(f"Train-auc:{train_auc:.4f}, Validate-auc:{validate_auc:.4f}")

In [ ]:
# Print the S3 path to the model artifact:
estimator.model_data

### MLflow 모델 레지스트리에 모델 등록
이제 훈련된 모델을 MLflow 모델 레지스트리에 등록합니다. 모델은 SageMaker 모델 레지스트리에도 자동으로 등록됩니다.
다음 스크립트는 최신 실험 실행에서 훈련된 모델을 등록한다는 점에 유의하세요. 선택한 훈련 옵션에 따라 모델은 로컬 훈련 실행, 스크립트 모드의 훈련 작업 또는 SageMaker 내장 알고리즘에서 나온 것일 수 있습니다.

In [ ]:
# get the last run in MLflow
last_run_id = mlflow.search_runs(
    experiment_ids=[mlflow.get_experiment_by_name(experiment_name).experiment_id], 
    max_results=1, 
    order_by=["attributes.start_time DESC"]
)['run_id'][0]

# construct the model URI
model_uri = f"runs:/{last_run_id}/model"

# register the model
registered_model_version = mlflow.register_model(model_uri, registered_model_name)

MLflow 등록된 모델에서 모델 메트릭도 확인할 수 있습니다:

In [ ]:
mlflow.get_run(registered_model_version.run_id).data.metrics

이제 SageMaker 모델 레지스트리를 살펴보겠습니다.

In [ ]:
sm = boto3.client('sagemaker')

In [ ]:
# get SageMaker model registry data for this model version
model_package_group_name = sm.list_model_package_groups(NameContains=registered_model_name)['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
sm_model_package = sm.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    )['ModelPackageSummaryList'][0]

In [ ]:
model_approval_status = 'PendingManualApproval'

# update SageMaker model version with mlflow cross-reference
sm.update_model_package(
        ModelPackageArn=sm_model_package['ModelPackageArn'],
        ModelApprovalStatus=model_approval_status,
        ApprovalDescription="created a new model version",
        CustomerMetadataProperties={
            "mlflow_model_name":registered_model_version.name,
            "mlflow_model_uri":model_uri,
            "mlflow_experiment_name":experiment_name,
        },
)

방금 MLflow와 SageMaker 모델 레지스트리 모두에 모델 버전을 등록했습니다. Studio나 MLflow UI를 통해, 그리고 SageMaker나 MLflow API를 통해 모델 레지스트리의 모든 등록된 모델에 액세스할 수 있습니다. 예를 들어, 아래 셀에서 생성된 링크를 클릭하여 Studio UI의 모델 레지스트리에서 모델을 엽니다.

In [ ]:
# Show the model registry link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}/versions">the model package group</a> in the Studio UI</b>'.format(
            domain_id, region, model_package_group_name))
)

### 선택사항: warm 풀로 훈련 작업 시작 시간 단축

<div class="alert alert-info">이 섹션은 선택사항입니다 – 워크샵의 추가 과정에는 필요하지 않습니다. <b>AWS에서 제공하는 워크샵 계정에서는 이 섹션을 실행하지 마세요.</b>.</div>

모델을 훈련할 때마다 새로운 임시 컴퓨팅 클러스터를 사용하는 대신, 지정된 기간 동안 모든 작업 후에 모델 훈련 하드웨어 인스턴스를 웜 상태로 유지할 수 있습니다. 자세한 내용은 [SageMaker Training Managed Warm Pools를 사용하여 ML 모델 훈련 작업 시작 시간을 최대 8배 단축](https://aws.amazon.com/about-aws/whats-new/2022/09/reduce-ml-model-training-job-startup-time-8x-sagemaker-training-managed-warm-pools/)을 참조하세요. 웜 풀을 사용하기로 선택하면 keep-alive 기간 동안 인스턴스와 EBS 볼륨에 대해 요금이 청구됩니다.
훈련 API에 대한 자세한 내용은 Amazon SageMaker 개발자 가이드의 [SageMaker Managed Warm Pools를 사용한 훈련](https://docs.aws.amazon.com/sagemaker/latest/dg/train-warm-pools.html)을 참조하세요.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">웜 풀 기능을 사용하려면 필요한 인스턴스 유형에 대한 해당 웜 풀 할당량이 0보다 큰 값으로 설정되어 있어야 합니다.
    <br>
    <br>
    웜 풀 할당량이 0으로 설정되어 있으므로 AWS에서 제공하는 워크샵 계정에서는 이 섹션을 실행하지 마세요.
    다음 섹션에서는 훈련 인스턴스 유형에 대한 할당량 값을 확인합니다.
    </p>
</div>

In [ ]:
def check_quota(quota_code, min_v):
    r = quotas_client.get_service_quota(
        ServiceCode="sagemaker",
        QuotaCode=quota_code,
    )
    
    q = r["Quota"]["Value"]
    n = r["Quota"]["QuotaName"]

    if q < min_v:
        print (
            f"WARNING: Your quota {q} for {n} is less than required value of {min_v}"
        )
    else:
        print(
            f"SUCCESS: Your quota {q} for {n} is equal or more than required value of {min_v}"
        )

In [ ]:
quotas_client = boto3.client("service-quotas")
                      
quotas = {
    "ml.m5.large": ["L-2DD73636", 1],
    "ml.m5.xlarge": ["L-0BEF44E8", 1],
    "ml.m5.2xlarge": ["L-1686EE8B", 1],
}
     
check_quota(quotas[train_instance_type][0], quotas[train_instance_type][1])

#### SageMaker 웜 풀로 훈련
이 기능을 사용하여 웜 풀을 사용한 XGBoost 훈련을 실행해보겠습니다.
이전 작업에서 프로비저닝된 인프라를 재사용하기 위한 훈련 작업의 일치 속성에 주목하세요: [일치 기준](https://docs.aws.amazon.com/sagemaker/latest/dg/train-warm-pools.html#train-warm-pools-matching-criteria)

웜 풀을 생성하려면 `Estimator` 구성에서 `KeepAlivePeriodInSeconds` 매개변수를 0보다 큰 값으로 설정해야 합니다.

In [ ]:
# Instantiate an XGBoost estimator object
warm_pool_estimator = sagemaker.estimator.Estimator(
    image_uri=training_image,  # XGBoost algorithm container
    instance_type=train_instance_type,  # type of training instance
    instance_count=train_instance_count,  # number of instances to be used
    role=sm_role,  # IAM execution role to be used
    max_run=20 * 60,  # Maximum allowed active runtime
    # use_spot_instances=True,  # Use spot instances to reduce cost
    # max_wait=30 * 60,  # Maximum clock time (including spot delays)
    output_path=output_s3_url, # S3 location for saving the training result
    sagemaker_session=session, # Session object which manages interactions with SageMaker API and AWS services
    base_job_name="from-idea-to-prod-training", # Prefix for training job name
    keep_alive_period_in_seconds=1800, # use the warm pool feature
)

In [ ]:
training_inputs = {'train': s3_input_train, 'validation': s3_input_validation}

다른 하이퍼파라미터로 `estimator.fit()`을 여러 번 연속으로 호출하여 훈련 작업을 실행합니다. 초기 훈련 작업은 SageMaker가 필요한 컴퓨팅 인프라를 프로비저닝하므로 "콜드 스타트"됩니다. 이 작업이 완료되면 인프라는 `KeepAlivePeriodInSeconds` 기간 동안 활성 상태로 유지됩니다. 웜 풀은 재사용을 위한 일치하는 훈련 작업을 식별하거나 지정된 `KeepAlivePeriodInSeconds`를 초과하여 종료될 때까지 `Available` 상태를 유지합니다.

다음 셀에서 생성된 링크를 클릭하여 SageMaker 콘솔에서 작업 실행을 추적합니다:

In [ ]:
# Show the training jobs link
display(
    HTML('<b>See SageMaker <a target="top" href="https://{}.console.aws.amazon.com/sagemaker/home?region={}#/jobs/">training jobs</a></b>'.format(
            region, region))
)

In [ ]:
# Start a new experiment to log execution times for each estimator fit
suffix = strftime('%d-%H-%M-%S', gmtime())
mlflow.set_experiment(experiment_name=f"from-idea-to-prod-warm-pools-{suffix}")

with mlflow.start_run(
    run_name=f"container-warm-pools-{suffix}",
    description="warm pools experiment in the notebook 02") as parent_run:
    # run the training job five times
    for i, d in enumerate([2, 3, 5, 10, 20]):
        print(f"Fit estimator with max_depth={d}")
    
        warm_pool_estimator.set_hyperparameters(
            num_round=50, # the number of rounds to run the training
            max_depth=d, # maximum depth of a tree
            objective="binary:logistic",
            eval_metric="auc", # evaluation metrics for validation data
            early_stopping_rounds=10, # the model trains until the validation score stops improving
        )

        with mlflow.start_run(
            run_name=f"max_depth={d}",
            description=f"Fit estimator with max_depth={d}",
            nested=True) as child_run:
            mlflow.log_params(warm_pool_estimator.hyperparameters())
            
            warm_pool_estimator.fit(
                training_inputs,
                wait=True,
                logs=False,
            )
            
            mlflow.log_metrics({i['metric_name'].replace(':', '_'):i['value'] for i in warm_pool_estimator.training_job_analytics.dataframe().iloc})

[SageMaker 훈련 작업 콘솔](https://console.aws.amazon.com/sagemaker/home?#/jobs)로 이동하여 훈련 작업 목록을 검사하면 이 훈련 작업에 웜 풀이 사용되었는지 확인할 수 있습니다:

![](img/warm-pools-training-jobs.png)

첫 번째 훈련 작업은 몇 분 정도 걸리지만, 모든 후속 작업은 동일한 컴퓨팅 인스턴스를 재사용하여 몇 초 만에 완료됩니다. 웜 풀 상태와 남은 시간도 확인할 수 있습니다.

MLflow 실험 목록을 열고 `from-idea-to-prod-warm-pools-<timestamp>` 실험을 선택할 수도 있습니다. 기록된 실행 시간을 보면 첫 번째 훈련은 약 2분이 걸렸고 모든 후속 실행은 40초 미만이 걸렸음을 알 수 있습니다:

![](img/warm-pools-mlflow.png)

## 모델 배포 및 테스트
모든 훈련 작업은 Amazon S3의 지정된 위치에 모델 패키지를 저장했습니다.

모델을 [실시간 엔드포인트](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html)로 배포할 수 있으며, 이는 단순히 하나의 [함수 호출](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator.deploy)입니다. 또는 대용량 데이터셋에 대한 레이블을 예측하기 위해 [배치 변환](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)을 생성할 수도 있습니다. [ModelBuilder](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-modelbuilder-creation.html) 클래스를 사용하여 MLflow 모델 레지스트리에서 엔드포인트로 모델을 배포할 수도 있습니다.

### 테스트 데이터 준비
S3에서 준비된 테스트 데이터셋을 노트북으로 다운로드합니다:

In [ ]:
!aws s3 cp $test_s3_url/test_x.csv tmp/test_x.csv
!aws s3 cp $test_s3_url/test_y.csv tmp/test_y.csv

### 노트북에서 로컬 추론
먼저 MLflow 모델 레지스트리에서 등록된 모델을 사용하여 로컬에서 추론을 실행합니다.

In [ ]:
# get the model from the MLflow model registry
model_uri = registered_model_version.source
model = mlflow.xgboost.load_model(model_uri)

In [ ]:
import xgboost as xgb
from numpy import loadtxt

test_x = loadtxt("tmp/test_x.csv", delimiter=",")
test_y = loadtxt("tmp/test_y.csv", delimiter=",")

dtest = xgb.DMatrix(test_x)

predictions = np.array(model.predict(dtest), dtype=float).squeeze()
predictions

In [ ]:
pd.crosstab(
    index=test_y,
    columns=np.round(predictions), 
    rownames=['actuals'], 
    colnames=['predictions']
)

### SageMaker 엔드포인트를 사용한 실시간 추론

이 섹션에서는 실시간 [SageMaker 엔드포인트](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html)를 생성합니다.

훈련된 추정기에서 모델을 배포하는 가장 쉬운 방법은 [`Estimator.deploy()`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy) 메서드를 사용하는 것입니다.

#### 로컬 배포

로컬에서 훈련된 추정기에서도 예측기를 배포할 수 있습니다. 이 경우 SageMaker 엔드포인트도 노트북의 알고리즘 컨테이너에 로컬로 배포됩니다.

In [ ]:
endpoint_name_from_local_mode_estimator = f"from-idea-to-prod-endpoint-estimator-{strftime('%d-%H-%M-%S', gmtime())}"

In [ ]:
# This creates a local SageMaker endpoint
predictor_from_local_mode_estimator = xgb_script_mode_local.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name_from_local_mode_estimator,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer(),
)

#### 예측

In [ ]:
test_x = loadtxt("tmp/test_x.csv", delimiter=",")
test_y = loadtxt("tmp/test_y.csv", delimiter=",")

In [ ]:
predictions = np.array(predictor_from_local_mode_estimator.predict(test_x), dtype=float).squeeze()
predictions

In [ ]:
test_auc = roc_auc_score(test_y, predictions)
print(f"Test-auc: {test_auc:.4f}")

In [ ]:
# delete the endpoint
predictor_from_local_mode_estimator.delete_endpoint(delete_endpoint_config=True)

#### SageMaker 인스턴스에 배포

In [ ]:
# take the estimator that you trained in the previous section
xgb_script_mode_managed.latest_training_job.job_name

In [ ]:
# create a unique endpoint name
endpoint_name_from_script_mode_estimator = f"from-idea-to-prod-endpoint-estimator-{strftime('%d-%H-%M-%S', gmtime())}"

In [ ]:
predictor_from_script_mode_estimator = xgb_script_mode_managed.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    wait=False,  # Don't wait in this call, but wait in the waiter in the next cell
    # Turn on data capture, in case you want to experiment with monitoring:
    data_capture_config=sagemaker.model_monitor.DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket_name}/{bucket_prefix}/data-capture",
    ),
    endpoint_name=endpoint_name_from_script_mode_estimator,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer(),
)

In [ ]:
# Wait until the endpoint has the status InService
waiter = session.sagemaker_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name_from_script_mode_estimator)

SageMaker 콘솔에서 배포된 엔드포인트를 확인할 수 있습니다 – 아래 셀에서 생성된 링크를 클릭하세요:

In [ ]:
# Show the inference endpoint link
display(
    HTML('<b>See the SageMaker <a target="top" href="https://{}.console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">inference endpoint</a></b>'.format(
            region, region, endpoint_name_from_script_mode_estimator))
)

#### 예측

In [ ]:
test_x = loadtxt("tmp/test_x.csv", delimiter=",")
test_y = loadtxt("tmp/test_y.csv", delimiter=",")

In [ ]:
predictions = np.array(predictor_from_script_mode_estimator.predict(test_x), dtype=float).squeeze()
predictions

In [ ]:
pd.crosstab(
    index=test_y,
    columns=np.round(predictions), 
    rownames=['actuals'], 
    colnames=['predictions']
)

In [ ]:
test_auc = roc_auc_score(test_y, predictions)
print(f"Test-auc: {test_auc:.4f}")

#### MLflow 실행에 차트 저장
[`mlflow.log_figure()`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_figure) 로깅 메서드를 사용하여 [matplotlib](https://matplotlib.org/3.3.4/api/_as_gen/matplotlib.figure.Figure.html) 또는 [plotly](https://plotly.com/python-api-reference/generated/plotly.graph_objects.Figure.html) 그림을 실행에 저장할 수 있습니다.

In [ ]:
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

def plot_confusion_matrix(
    cm, class_names, title="Confusion matrix", cmap=plt.cm.Blues, normalize=False
):
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation="nearest", cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        ylim=(cm.shape[0] - 0.5, -0.5),
        xticklabels=class_names,
        yticklabels=class_names,
        title=title,
        ylabel="Ground truth label",
        xlabel="Predicted label",
    )

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = ".2f"
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,
                i,
                format(cm[i, j], fmt),
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
            )
    fig.tight_layout()
    return ax, fig

In [ ]:
class_names = ["no", "yes"]
confusion_matrix = metrics.confusion_matrix(test_y, np.round(predictions))
ax, fig = plot_confusion_matrix(confusion_matrix, class_names)

In [ ]:
print(f"Log confusion matrix to the model {registered_model_version.name} version {registered_model_version.version}")
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_id=registered_model_version.run_id):
    mlflow.log_figure(fig, "confusion_matrix.png")

In [ ]:
# get the presigned url to open the MLflow UI
presigned_url = sm.create_presigned_mlflow_tracking_server_url(
    TrackingServerName=mlflow_name,
    ExpiresInSeconds=60,
    SessionExpirationDurationInSeconds=1800
)['AuthorizedUrl']

mlflow_run_link = f"{presigned_url.split('/auth')[0]}/#/experiments/1/runs/{registered_model_version.run_id}/artifacts"

In [ ]:
# first open the MLflow UI - you can close a new opened window
display(Javascript('window.open("{}");'.format(presigned_url)))

In [ ]:
# second open the run page in the MLflow UI
display(Javascript('window.open("{}");'.format(mlflow_run_link)))

#### 정리

In [ ]:
# delete the endpoint to avoid cost
predictor_from_script_mode_estimator.delete_endpoint(delete_endpoint_config=True)

#### 선택사항: 내장 추정기에서 배포
<div class="alert alert-info">이 섹션은 선택사항입니다 – 워크샵의 추가 과정에는 필요하지 않습니다. 이 세션을 실행하려면 이전 섹션 <b>선택사항: SageMaker 내장 알고리즘으로 훈련</b>에서 SageMaker 내장 알고리즘 훈련 작업을 실행해야 합니다.</div>

훈련된 추정기 객체가 있다면 추정기의 [`deploy()`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator.deploy) 메서드를 사용하여 한 줄의 코드로 모델을 배포할 수 있습니다. 이 섹션에서는 이를 수행하는 방법을 보여줍니다.

In [ ]:
# take the XGBoost estimator that you trained in the previous section
estimator.latest_training_job.job_name

In [ ]:
# create a real-time endpoint from the trained estimator
endpoint_name_from_estimator = f"from-idea-to-prod-endpoint-estimator-{strftime('%d-%H-%M-%S', gmtime())}"

predictor_from_estimator = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    wait=True,  # Remember, predictor.predict() won't work until deployment finishes!
    # Turn on data capture, in case you want to experiment with monitoring:
    data_capture_config=sagemaker.model_monitor.DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket_name}/{bucket_prefix}/data-capture",
    ),
    endpoint_name=endpoint_name_from_estimator,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer(),
)

In [ ]:
# load test data as CSV files
test_x = pd.read_csv("tmp/test_x.csv", header=None)
test_y = pd.read_csv("tmp/test_y.csv", names=['y'])

In [ ]:
# Predict using the real-time endpoint predictor
predictions = np.array(predictor_from_estimator.predict(test_x.values), dtype=float).squeeze()
predictions

In [ ]:
test_results = pd.concat(
    [
        pd.Series(predictions, name="y_pred", index=test_x.index),
        test_x,
    ],
    axis=1,
)
test_results.head()

In [ ]:
pd.crosstab(
    index=test_y['y'].values,
    columns=np.round(predictions), 
    rownames=['actuals'], 
    colnames=['predictions']
)

In [ ]:
test_auc = roc_auc_score(test_y, test_results["y_pred"])
print(f"Test-auc: {test_auc:.4f}")

#### 정리

In [ ]:
# delete the endpoint to avoid cost
predictor_from_estimator.delete_endpoint(delete_endpoint_config=True)

## 선택사항: 배치 변환
대용량 데이터셋에서 오프라인 예측을 실행하거나 실시간 엔드포인트가 필요하지 않은 경우 SageMaker [배치 변환](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)을 사용할 수 있습니다. 이 섹션에서는 [SageMaker 배치 변환](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) 사용 방법을 보여줍니다.

<div class="alert alert-info"><b>선택사항: SageMaker 내장 알고리즘으로 훈련</b> 섹션에서 SageMaker 내장 XGBoost로 추정기를 훈련한 경우에만 이 세션을 실행하세요.
</div>

In [ ]:
transform_s3_url = f"s3://{bucket_name}/{bucket_prefix}/transform"
model_name = f"from-idea-to-prod-transform-{strftime('%d-%H-%M-%S', gmtime())}"

<div class="alert alert-info"> 💡 변환기를 생성하려면 <b>옵션 1</b> 또는 <b>옵션 2</b> 중 하나를 사용하세요
</div>

### 옵션 1: 훈련된 추정기에서 배치 변환기 생성
훈련된 추정기가 있는 경우 [`EstimatorBase.transformer()`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.transformer)를 사용하여 추정기에 대한 변환기를 생성할 수 있습니다.

In [ ]:
try:
    trained_estimator = estimator
except NameError:
    print("You don't have a trained estimator. Run SageMaker remote training or use a training job name.")

trained_estimator

In [ ]:
transformer = trained_estimator.transformer(
    instance_count=1,
    instance_type=train_instance_type,
    accept="text/csv",
    role=sm_role,
    output_path=transform_s3_url,
    model_name=model_name,
)

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">옵션 2를 건너뛰고 <b>변환 작업 실행</b> 섹션으로 이동하세요.
    </p>
</div>

### 옵션 2: 훈련 작업에서 모델 로드
또는 훈련 작업에서 생성된 모델 아티팩트에서 모델을 로드할 수 있습니다. 해당 모델로 변환기를 생성합니다.

In [ ]:
if training_job_name is None:
    print("You don't have saved trained job name. Run SageMaker built-in algorithm training or use a trained estimator.")

In [ ]:
model = session.create_model_from_job(
    training_job_name=training_job_name, 
    name=model_name,
)

In [ ]:
transformer = sagemaker.transformer.Transformer(
    model_name=model,
    instance_count=1,
    instance_type=train_instance_type,
    accept="text/csv",
    assemble_with="Line",
    output_path=transform_s3_url,
    base_transform_job_name="from-idea-to-prod-trasform",
    sagemaker_session=session,
)

In [ ]:
!aws s3 cp ./tmp/test_x.csv {test_s3_url}/test_x.csv

In [ ]:
test_s3_url

### 변환 작업 실행



In [ ]:
transform_job_name = f"from-idea-to-prod-transform-{strftime('%d-%H-%M-%S', gmtime())}"

transformer.transform(    
    data=f"{test_s3_url}/test_x.csv",
    content_type="text/csv",
    split_type="Line", 
    job_name=transform_job_name,
    wait=True,
)

In [ ]:
while sm.describe_transform_job(
        TransformJobName=transformer._current_job_name
    )["TransformJobStatus"] != "Completed":
    time.sleep(10)
    print(f"Wait until {transformer._current_job_name} completed")

In [ ]:
transformer.output_path

### 예측 평가

In [ ]:
!aws s3 ls {transformer.output_path}/

In [ ]:
!aws s3 cp {transformer.output_path}/test_x.csv.out tmp/predictions.csv
!aws s3 cp $test_s3_url/test_y.csv tmp/test_y.csv

In [ ]:
predictions = pd.read_csv("tmp/predictions.csv", names=["y_prob"])
test_y = pd.read_csv("tmp/test_y.csv", names=['y'])

#### Crosstab

In [ ]:
pd.crosstab(
    index=test_y['y'].values,
    columns=np.array(np.round(predictions), dtype=float).squeeze(), 
    rownames=['actuals'], 
    colnames=['predictions']
)

In [ ]:
test_auc = roc_auc_score(test_y, predictions)
print(f"Test-auc: {test_auc:.4f}")

#### ROC curve

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(test_y, predictions)
roc_auc = metrics.auc(fpr, tpr)
roc_display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='Holdout/Test Data - ROC curve')
roc_display.plot()
plt.show()

#### Confusion matrix

In [ ]:
class_names = ["no", "yes"]
confusion_matrix = metrics.confusion_matrix(test_y, np.round(predictions))
ax, fig = plot_confusion_matrix(confusion_matrix, class_names)

#### Precision-recall curve

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

prec, recall, _ = precision_recall_curve(test_y, predictions)
average_precision= metrics.average_precision_score(test_y, predictions)
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall, average_precision=average_precision, estimator_name='Holdout/Test Data - AUPRC curve')
pr_display.plot()
plt.show()

## MLflow UI로 실험 및 모델 레지스트리 탐색
MLflow UI에서 기록된 모든 메트릭, 매개변수 및 아티팩트를 확인할 수 있습니다. MLflow UI를 시작하려면 Studio UI의 **Application** 창에서 **MLflow**를 선택하고, MLflow 서버를 선택한 다음 **Open MLflow**를 선택합니다:

![](img/mlflow-open.png)

MLflow UI에서 실험, 실행 및 등록된 모델을 탐색할 수 있습니다:

![](img/experiment-mlflow-2.png)

![](img/models-mlflow-2.png)

![](img/model-fig-mlflow.png)

---

## 선택사항: 하이퍼파라미터 최적화(HPO)
<div class="alert alert-info">이 섹션은 선택사항입니다 – 워크샵의 추가 과정에는 필요하지 않습니다. <b>정리</b> 섹션으로 이동할 수 있습니다.</div>

하이퍼파라미터 최적화(HPO)라고도 하는 [Amazon SageMaker 자동 모델 튜닝](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)은 지정한 알고리즘과 하이퍼파라미터 범위를 사용하여 데이터셋에서 많은 훈련 작업을 실행하여 정의된 목표 메트릭에 대해 최고 성능의 모델을 찾습니다. SageMaker HPO는 튜닝 전략으로 랜덤 검색, 베이지안 최적화 및 [하이퍼밴드](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html)를 지원합니다.

In [ ]:
suffix = strftime('%d-%H-%M-%S', gmtime())
hpo_experiment_name = f"from-idea-to-prod-hpo-{suffix}"
registered_hpo_model_name = f"from-idea-to-prod-hpo-model-{suffix}"

mlflow.set_experiment(hpo_experiment_name)

In [ ]:
# import required HPO objects
from sagemaker.tuner import (
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
    IntegerParameter,
)

In [ ]:
# set up hyperparameter ranges
hp_ranges = {
    "min_child_weight": ContinuousParameter(1, 10),
    "max_depth": IntegerParameter(1, 10),
    "alpha": ContinuousParameter(0, 5),
    "eta": ContinuousParameter(0, 1),
    "colsample_bytree": ContinuousParameter(0, 1),
}

# set up the objective metric
objective = "validation:auc"

# set up the estimator in script mode
hpo_estimator = XGBoost(
    entry_point='train.py',
    source_dir='./training',
    framework_version="1.7-1",  
    hyperparameters=hyperparams,
    role=sm_role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    output_path=output_s3_url,
    base_job_name="from-idea-to-prod-training",
    environment={
        'MLFLOW_TRACKING_ARN': mlflow_arn,
        'MLFLOW_EXPERIMENT_NAME': hpo_experiment_name,
        'USER': user_profile_name,
        'REGION': region,
    }
)

# instantiate a HPO object
tuner = HyperparameterTuner(
    estimator=hpo_estimator,  # the SageMaker estimator object
    hyperparameter_ranges=hp_ranges,  # the range of hyperparameters
    max_jobs=10,  # total number of HPO jobs
    max_parallel_jobs=3,  # how many HPO jobs can run in parallel
    strategy="Bayesian",  # the internal optimization strategy of HPO
    objective_metric_name=objective,  # the objective metric to be used for HPO
    objective_type="Maximize",  # maximize or minimize the objective metric
    base_tuning_job_name="from-idea-to-prod-hpo",
    early_stopping_type="Auto",
)

### HPO 실행
이제 HPO 작업을 실행합니다. 완료하는 데 약 10분이 걸립니다.

In [ ]:
tuner.fit(
    {"train": s3_input_train, "validation": s3_input_validation},
)

In [ ]:
tuner.describe()['HyperParameterTuningJobStatus']

SageMaker 콘솔에서 HPO 작업의 세부 정보를 보려면 아래 셀에서 생성된 링크를 클릭하세요:

In [ ]:
# Show the HPO job
display(
    HTML('<b>See the SageMaker <a target="top" href="https://{}.console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/{}">HPO job</a></b>'.format(
            region, region, tuner.latest_tuning_job.job_name))
)

### HPO 결과 확인
이제 결과를 가져와서 최적 훈련 작업을 확인합니다.

In [ ]:
best_training_job = tuner.describe()['BestTrainingJob']

In [ ]:
best_training_job

### 모델 레지스트리에 최적 HPO 모델 등록
하이퍼파라미터, 메트릭 및 모델 아티팩트와 함께 최적 HPO 모델을 MLflow 모델 레지스트리에 등록합니다.

In [ ]:
# find the run with the best HPO model
best_hpo_model_run_id = mlflow.search_runs(
    experiment_ids=[mlflow.get_experiment_by_name(hpo_experiment_name).experiment_id], 
    filter_string=f"tags.mlflow.source.name LIKE '%{best_training_job['TrainingJobArn'].split('/')[-1]}%'",
)['run_id'][0]

In [ ]:
best_hpo_model_run_id

In [ ]:
# Register the model
model_uri = f"runs:/{best_hpo_model_run_id}/model"
registered_hpo_model_version = mlflow.register_model(model_uri, registered_hpo_model_name)

다음 단계로 최적 HPO 모델을 배포하고 테스트해야 합니다. 아래 옵션 중 하나를 선택할 수 있습니다.

### 최적 HPO 모델로 로컬 추론 실행

In [ ]:
# get the model from the MLflow model registry
model_uri = registered_hpo_model_version.source
model = mlflow.xgboost.load_model(model_uri)

# load data
test_x = loadtxt("tmp/test_x.csv", delimiter=",")
test_y = loadtxt("tmp/test_y.csv", delimiter=",")

# predict
dtest = xgb.DMatrix(test_x)

predictions = np.array(model.predict(dtest), dtype=float).squeeze()
predictions

In [ ]:
pd.crosstab(
    index=test_y,
    columns=np.round(predictions), 
    rownames=['actuals'], 
    colnames=['predictions']
)

In [ ]:
test_auc = roc_auc_score(test_y, predictions)
print(f"Test-auc: {test_auc:.4f}")

모델 메트릭에는 약간의 개선만 있습니다. 이는 XGBoost 모델이 이미 한계에 도달했음을 나타낼 수 있습니다. 이 사용 사례에서 예측 정확도를 향상시키기 위해 다른 모델 유형을 탐색해볼 수 있습니다.

---

## 정리
요금을 피하기 위해 생성했지만 아직 제거하지 않은 배포된 엔드포인트를 제거하세요.

In [ ]:
sm_client = boto3.client('sagemaker')

In [ ]:
endpoints = sm_client.list_endpoints()["Endpoints"]

In [ ]:
endpoints_to_delete = []

for ep in endpoints:
    print(f"Are you sure you want to delete this endpoint: {ep['EndpointName']}? (y/n)")
    choice = input()
    if choice == 'y':
        endpoints_to_delete.append(ep['EndpointName'])
        
print(f"*********** THESE ENDPOINTS WILL BE DELETED ***********")
print('\n'.join(endpoints_to_delete))
print(f"*******************************************************")

In [ ]:
for ep in endpoints_to_delete:
    try:
        print(f"Deleting the endpoint: {ep}:{sm_client.delete_endpoint(EndpointName=ep)}")
    except Exception as e:
        print(f"Exception in deleting {ep}:{e}")
        pass

## 3단계 계속하기
3단계 [노트북](03-sagemaker-pipeline.ipynb)을 엽니다.

## 실제 프로젝트를 위한 추가 개발 아이디어
- [실험 관리](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow.html)
- 노코드 또는 로우코드 시각적 데이터 처리 및 피처 엔지니어링 플로우를 생성하기 위해 [Amazon SageMaker Data Wrangler](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler.html) 사용
- 데이터에서 분석을 수행하고 자동화된 ML을 사용하여 모델을 구축하고 예측을 생성하기 위해 노코드 [SageMaker Canvas](https://docs.aws.amazon.com/sagemaker/latest/dg/canvas.html) 시도

## 추가 리소스
- [Using Docker containers with SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers.html)
- [How to create and use a custom SageMaker container: SageMaker hands-on workshop](https://sagemaker-workshop.com/custom/containers.html)
- [Amazon SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US)
- [Targeting Direct Marketing with Amazon SageMaker XGBoost](https://github.com/aws-samples/amazon-sagemaker-immersion-day/blob/master/processing_xgboost.ipynb)
- [Train a Machine Learning Model](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-train-a-model/)
- [Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [Amazon SageMaker 101 Workshop](https://catalog.us-east-1.prod.workshops.aws/workshops/0c6b8a23-b837-4e0f-b2e2-4a3ffd7d645b/en-US)
- [Amazon SageMaker 101 Workshop code repository](https://github.com/aws-samples/sagemaker-101-workshop)
- [Amazon SageMaker with XGBoost and Hyperparameter Tuning for Direct Marketing predictions](https://github.com/aws-samples/sagemaker-101-workshop/blob/main/builtin_algorithm_hpo_tabular/SageMaker%20XGBoost%20HPO.ipynb)

# kernel 정지

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>